In [1]:
import matplotlib.pyplot as plt
import librosa
from IPython.display import Audio
import pandas as pd
import numpy as np
import torch
from bc import bit_cypher
import os
import random
from sklearn.cluster import KMeans


In [2]:
files = os.walk('../data/train_files_48k')
sr = 48000

In [26]:
files = []
for root,dirs,file in os.walk('../data/train_files_48k'):
    for f in file:
        path = os.path.join(root,f)
        files.append(path)



In [27]:
files = random.sample(files,10000)

In [28]:
x = torch.stack([torch.tensor(librosa.load(x,sr=sr)[0]) for x in files])

In [6]:
print('num unique before quantize')
pd.Series(x.flatten()).nunique()

num unique before quantize


97900333

In [29]:
kmeans = KMeans(n_clusters=511)
kmeans.fit(x[0:10].reshape(-1, 1))

KMeans(n_clusters=511)

In [30]:
idx = torch.stack(([torch.tensor(kmeans.predict(t.reshape(-1,1))) for t in x])) # idxs
 
centroids = kmeans.cluster_centers_.flatten() # vocab
 

In [42]:
x.shape

torch.Size([10000, 240000])

In [31]:
centroid2embed = {x:i for i,x in enumerate(pd.Series(idx.flatten()).value_counts().index.to_list())} # index of centroid to rank (idx of bit cipher)

In [32]:
embedding_idx = [centroid2embed[int(x)] for x in idx[0]] 

In [33]:
u,v,vocab_size = bit_cypher(9)

In [34]:
embeddings = v[embedding_idx]

In [58]:
embeddings


tensor([[0.0000, 0.0000, 0.2500,  ..., 0.0000, 0.2500, 0.2500],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2500, 0.0000],
        [0.2500, 0.0000, 0.2500,  ..., 0.0000, 0.2500, 0.0000],
        ...,
        [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])